In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# my virenv : py37NLP

import os
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
pwd

In [ ]:
trainData = pd.read_csv('/kaggle/input/feedback-prize-2021/train.csv')
trainData

In [ ]:
trainData.info()

In [ ]:
# load dataset
def load_df():
    train_names, train_texts = [], []
    for f in tqdm(list(os.listdir('/kaggle/input/feedback-prize-2021/train'))):
        train_names.append(f.replace('.txt', ''))
        train_texts.append(open('/kaggle/input/feedback-prize-2021/train/' + f, 'r', encoding='UTF-8').read())
    train_text_df = pd.DataFrame({'id': train_names, 'text': train_texts})
    return train_text_df

In [ ]:
df = load_df()
df.head()

In [ ]:
# Check the value
df_ = df.query("id == 'DBF7EB6A9E02'")['text'].values[0]
df_

In [ ]:
tr_df = trainData.query("id == 'DBF7EB6A9E02'")['discourse_text'].values[0]
tr_df

Since the length of the sentence to be analyzed is too long to be processed as BERT, the method of comparing and analyzing the input sentence after summarizing it is applied.


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
#from transformers import pipeline 
# use bart in pytorch
#summarizer = pipeline("summarization")
#ptorch = summarizer("An apple a day, keeps the doctor away", min_length=5, max_length=20)

#print(ptorch)

In [ ]:
df.head()

import pickle
df.to_pickle("tr_data_bert.pkl")

In [ ]:
# load data
import pickle
with open('../input/studentevaluation-preprocessed-datasets/tr_data_bert.pkl', 'rb') as f:
    Trdata = pickle.load(f)

In [ ]:
Trdata.head()


In [ ]:
# Preprocess...

def clean_text(text, remove_stopwords=True, stem_words=False, lemma=True):
    text = str(text).lower().split()
    
    if remove_stopwords:
        stops = stopwords.words("english")
        text = [x for x in text if not x in stops]
        
    text = ' '.join(text)
    
    text = re.sub(r"[-()\"#/<>!@&;*:<>{}`'+=~%|.!?,_]", " ", text)
    text = re.sub(r"\]", " ", text)
    text = re.sub(r"\[", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"0x00", "", text)
    text = re.sub(r"summary_text", "", text)
    text = re.sub(r"summary", "", text)
    text = re.sub(r"text", "", text)
    
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stem_words = [stemmer.stem(x) for x in text]
        text = " ".join(text)
        
    if lemma:
        text = text.split()
        lem = WordNetLemmatizer()
        lemmatized = [lem.lemmatize(x, "v") for x in text]
        text = " ".join(text)
        
    return text

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('omw-1.4')

df['t_summary'] = Trdata.text_summary.apply(clean_text)

In [ ]:
df.head()

In [ ]:
trainData_ = pd.merge(trainData,df)
trainData_[40:80]

In [ ]:
# import pickle
# trainData_.to_pickle("tr_data_vol4.pkl")

In [ ]:
# Load data
import pickle
with open('../input/train-data-vol4/tr_data_vol4.pkl', 'rb') as f:
    Trdata = pickle.load(f)

In [ ]:

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [ ]:
Trdata

In [ ]:
data_tr = Trdata[['id', 'text', 'discourse_text', 't_summary', 'discourse_type']]
data_tr[:5]

In [ ]:
# shaffle datasets
data = data_tr.sample(frac=1).reset_index(drop=True)
data

In [ ]:
num_classes=len(data.discourse_type.unique())
num_classes

In [ ]:
!pip uninstall tensorflow --yes

In [ ]:
!pip install tensorflow-gpu==1.15

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
!pip install bert-tensorflow

In [ ]:
!pip install bert

In [ ]:
!pip install bert-for-tf2

In [ ]:
import bert
from bert import modeling

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import os
import numpy as np
import pandas as pd
import datetime
import sys
import zipfile

#import modeling
from bert import optimization
from bert import run_classifier
from bert import tokenization

from bert import bert_tokenization

#from tokenization import FullTokenizer
from sklearn.preprocessing import LabelEncoder
#import tensorflow as tf


from sklearn.model_selection import train_test_split

import tensorflow_hub as hub
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [ ]:
sess = tf.Session()
 
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

# max_seq_length 토큰 최대 입력길이로,  128이상은 안됨. ㅠㅠ 그래서 이것을 해결해야 함. 
# LONGFORMER로 해야하지만 에러가 발생. 그래서 토큰을 잘라서 적용하는 방법을 생각해봄 

# 하지만 참조할 것  https://github.com/google-research/bert/blob/master/README.md

# H=768 모델 사용 '12/768 (BERT-Base)''
max_seq_length = 512

In [ ]:
# 'Claim' 'Evidence' 'Position' 'Counterclaim' 'Lead'
# 'Concluding Statement' 'Rebuttal'


data['label'] = data['discourse_type'].map({'Claim':0,'Evidence':1, 'Position':2,'Counterclaim':3, 'Lead':4, 'Concluding Statement':5, 'Rebuttal':6})
data.head()

In [ ]:
cols = ['t_summary', 'discourse_text']
data['text_concat'] = data[cols].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)
data

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.2)

In [ ]:
train_df.head()

In [ ]:
tr_df_sample = train_df.query("id == 'D0C7DA676408'")['text_concat'].values[0]
tr_df_sample

In [ ]:
train_df, val_df = train_test_split(test_df, test_size=0.5)

In [ ]:
label_encoder = LabelEncoder().fit(pd.concat([train_df['discourse_type'], val_df['discourse_type']]))

In [ ]:
# Check Classes
print(label_encoder.classes_)

In [ ]:
test_df

In [ ]:
# data concat full text and discourse_text
X_train_val, X_test = pd.concat([train_df['text_concat'], val_df['text_concat']]).values, test_df['text_concat'].values
 
y_train_val = label_encoder.fit_transform(pd.concat([train_df['discourse_type'], val_df['discourse_type']]))

In [ ]:
test_df['text_concat'].values

In [ ]:
X_train_val

In [ ]:
X_test

In [ ]:
y_train_val

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
        X_train_val,y_train_val, test_size=0.1, random_state=0, stratify = y_train_val
        )

In [ ]:
y_val

In [ ]:
 # max_seq_length의 길리만큼 끊어서 넘파이 어레이로 만듬. 이것을 가지로 학습, 검증, 테스트를 수행 
train_text = X_train
train_text = [' '.join(t.split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = y_train
 
val_text = X_val
val_text = [' '.join(t.split()[0:max_seq_length]) for t in val_text]
val_text = np.array(val_text, dtype=object)[:, np.newaxis]
val_label = y_val
 
test_text = X_test
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

In [ ]:
train_text

In [ ]:
test_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import re
import numpy as np
from tqdm import tqdm_notebook
from tensorflow import keras 
from tensorflow.keras.layers import Layer

In [ ]:
class BertLayer(Layer):
    
     #bert_output = BertLayer(n_fine_tune_layers=n_fine_tune, tf_hub = tf_hub, output_representation = 'mean_pooling', is_trainable = True, output_size = 768, supports_masking = True)(bert_inputs)
     #TypeError: __init__() missing 3 required positional arguments: 'is_trainble', 'output_size', and 'supports_masking'
        
    # init
    def __init__(self, n_fine_tune_layers=10, tf_hub = None, output_representation = 'pooled_output', trainable = False, **kwargs):
        
        self.n_fine_tune_layers = n_fine_tune_layers
        self.is_trainble = trainable
        self.output_size = 768
        self.tf_hub = tf_hub
        self.output_representation = output_representation
        self.supports_masking = True
        
        super(BertLayer, self).__init__(**kwargs)

        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_fine_tune_layers' : self.n_fine_tune_layers,
            'tf_hub' : self.tf_hub,
            'output_representation' : self.output_representation,
            'is_trainble' : self.is_trainble,
            'output_size' : self.output_size,
            'supports_masking' : self.supports_masking,
        })        
        return config

    
    def build(self, input_shape):

        self.bert = hub.Module(
            self.tf_hub,
            trainable=self.is_trainble,
            name="{}_module".format(self.name)
        )
        
        
        variables = list(self.bert.variable_map.values())
        if self.is_trainble:
            # 1 first remove unused layers
            trainable_vars = [var for var in variables if not "/cls/" in var.name]
            
            
            if self.output_representation == "sequence_output" or self.output_representation == "mean_pooling":
                # 1 first remove unused pooled layers
                trainable_vars = [var for var in trainable_vars if not "/pooler/" in var.name]
                
            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
            
            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            # Add non-trainable weights
            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)
                
        else:
             for var in variables:
                self._non_trainable_weights.append(var)
                

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)
        
        if self.output_representation == "pooled_output":
            pooled = result["pooled_output"]
            
        elif self.output_representation == "mean_pooling":
            result_tmp = result["sequence_output"]
        
            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result_tmp, input_mask)
            
        elif self.output_representation == "sequence_output":
            
            pooled = result["sequence_output"]
       
        return pooled
    
    def compute_mask(self, inputs, mask=None):
        
        if self.output_representation == 'sequence_output':
            inputs = [K.cast(x, dtype="bool") for x in inputs]
            mask = inputs[1]
            
            return mask
        else:
            return None
        
        
    def compute_output_shape(self, input_shape):
        if self.output_representation == "sequence_output":
            return (input_shape[0][0], input_shape[0][1], self.output_size)
        else:
            return (input_shape[0][0], self.output_size)


In [ ]:
def build_model(max_seq_length, tf_hub, n_classes, n_fine_tune): 
    in_id = keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    #bert_output = BertLayer(n_fine_tune_layers=n_fine_tune, tf_hub = tf_hub, output_representation = 'mean_pooling', is_trainable = True)(bert_inputs)
    bert_output = BertLayer(n_fine_tune_layers=n_fine_tune, tf_hub = tf_hub, output_representation = 'mean_pooling', trainable = True)(bert_inputs)
    drop = keras.layers.Dropout(0.3)(bert_output)
    dense = keras.layers.Dense(256, activation='sigmoid')(drop)
    drop = keras.layers.Dropout(0.3)(dense)
    dense = keras.layers.Dense(64, activation='sigmoid')(drop)
    pred = keras.layers.Dense(n_classes, activation='softmax')(dense)
    
    model = keras.models.Model(inputs=bert_inputs, outputs=pred)
    
    #Adam = keras.optimizers.Adam(lr = 5e-05) ->   only not improved val_loss
    #Adam = keras.optimizers.Adam(lr = 0.0005) ->   not imporved at all
    #Adam = keras.optimizers.Adam(lr = 0.000005) -> val loss 0.87
    #sgd =  keras.optimizers.SGD(lr=0.00005, decay=1e-6, momentum=0.9, nesterov=True) -> need to increse Epoh over 60 + 
    #sgd =  keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    #sgd =  keras.optimizers.SGD(lr=0.00005, decay=1e-6, momentum=0.9, nesterov=True) # increse Epoh over 60 +
    #Adam = keras.optimizers.Adam(lr = 5e-05, epsilon=1e-08, decay = 0.01, clipnorm = 1.0) -> loss: 0.8296 - sparse_categorical_accuracy: 0.7596 - val_loss: 0.8668 - val_sparse_categorical_accuracy: 0.7360
    Adam = keras.optimizers.Adam(lr=1e-5, epsilon=1e-8)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam, metrics=['sparse_categorical_accuracy'])
    model.summary()

    return model

def initialize_vars(sess): #tf.compat.v1.keras.backend.set_session
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [ ]:
n_classes = len(label_encoder.classes_)
n_fine_tune_layers = 48
model = build_model(max_seq_length, bert_path, n_classes, n_fine_tune_layers)

# Instantiate variables
initialize_vars(sess)

In [ ]:
model.trainable_weights

In [ ]:
# Reference : https://github.com/dmis-lab/biobert/blob/master/run_classifier.py

class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module(tf_hub):
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(tf_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )
    
    return bert_tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)
    
    #print(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [ ]:

# Instantiate tokenizer  bert_tokenization.FullTokenizer
tokenizer = create_tokenizer_from_hub_module(bert_path)

# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
val_examples = convert_text_to_examples(val_text, val_label)
test_examples = convert_text_to_examples(test_text, val_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(val_input_ids, val_input_masks, val_segment_ids, val_labels
) = convert_examples_to_features(tokenizer, val_examples, max_seq_length=max_seq_length)

## The following code hans been annotated because it takes a lot of time to learn when executing the code.



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

BATCH_SIZE = 16


MONITOR = 'val_sparse_categorical_accuracy'
print('BATCH_SIZE is {}'.format(BATCH_SIZE))
e_stopping = EarlyStopping(monitor=MONITOR, patience=3, verbose=1, mode='max', restore_best_weights=True)
callbacks =  [e_stopping]

history = model.fit(
   [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data = ([val_input_ids, val_input_masks, val_segment_ids], val_labels),
    epochs = 50,
    verbose = 1,
    batch_size = BATCH_SIZE,
    callbacks= callbacks
)

In [ ]:
def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""
 
    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )
 
def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [ ]:
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [ ]:
prediction = model.predict([test_input_ids, test_input_masks, test_segment_ids], verbose = 1)

In [ ]:
preds = label_encoder.classes_[np.argmax(prediction, axis =1)]

In [ ]:
preds

In [ ]:
pd.DataFrame(preds, columns=['label']).to_csv('/kaggle/input/feedback-prize-2021/submission.csv',index_label='id')

In [ ]:
Save model
model.save('evauation.h5')

In [ ]:
# ../input/evaluation/evauation.h5

# 'n_fine_tune_layers' : self.n_fine_tune_layers,
# 'is_trainble' : self.is_trainble,
# 'output_size' : self.output_size,
# 'tf_hub' : self.tf_hub,
# 'output_representation' : self.output_representation,
# 'supports_masking' : self.supports_masking,
                
                
# from tensorflow import keras
# #model = keras.models.load_model('../input/evaluation/evauation.h5')
# savedMode = keras.models.load_model('../input/evaluation/evauation.h5', custom_objects={"build_model":build_model})

In [ ]:
submission = pd.read_csv('/kaggle/input/feedback-prize-2021/sample_submission.csv')
submission

In [ ]:
def load_df_test():
    test_names, test_texts = [], []
    for f in tqdm(list(os.listdir('test'))):
        test_names.append(f.replace('.txt', ''))
        test_texts.append(open('test/' + f, 'r', encoding='UTF-8').read())
    test_text_df = pd.DataFrame({'id': test_names, 'text': test_texts})
    return test_text_df

test_df_sub = load_df_test()
test_df_sub

In [ ]:
test_summary_list = []
for text in tqdm(test_df_sub['text']):
    try:
        ptorch_summary = summarizer(text, min_length=5, max_length=200)
    except:
        pass
    
    test_summary_list.append(ptorch_summary)
    
test_df_sub['text_summary'] = test_summary_list
test_df_sub

In [ ]:
test_df_sub['cleaned_summary'] = test_df_sub.text_summary.apply(clean_text)
test_df_sub

In [ ]:
from nltk.tokenize import sent_tokenize


In [ ]:
discouse_text_pre_list = []
for text in tqdm(test_df_sub['text']):
    sent_token = sent_tokenize(text)
    discouse_text_pre_list.append(sent_token)
    
test_df_sub['discourse_text'] = discouse_text_pre_list
test_df_sub

In [ ]:
test_df_sub['id']

In [ ]:
for each_id in tqdm(test_df_sub['id']):
    print(type(each_id))

In [ ]:
testList = test_df_sub.query("id == '0FB0700DAF44'")['discourse_text'].values[0]
testList[:3]


In [ ]:
# get_text = test_df_sub.query("id == '0FB0700DAF44'")['text'].values[0]
# get_text

In [ ]:
for each_id in tqdm(test_df_sub['id']):
    #print("each_id :", each_id)
    get_dis_text_list = test_df_sub.query("id == @each_id")['discourse_text'].values[0]
    get_text = test_df_sub.query("id == @each_id")['text'].values[0]
    get_cleaned_summary = test_df_sub.query("id == @each_id")['cleaned_summary'].values[0]
    for disc_text in get_dis_text_list:
        data_insert = {'id': each_id, 'text':get_text, 'cleaned_summary': get_cleaned_summary,'new_discourse_text':disc_text }
        test_df_sub = test_df_sub.append(data_insert, ignore_index=True)


In [ ]:
test_df_sub.head()

In [ ]:
test_df_sub[['id', 'text', 'cleaned_summary', 'new_discourse_text']][:20]

In [ ]:
new_test_df = pd.DataFrame(columns=['id', 'text', 'cleaned_summary', 'new_discourse_text'])
new_test_df

In [ ]:
new_test_df = pd.concat([new_test_df, test_df_sub[['id', 'text', 'cleaned_summary', 'new_discourse_text']]])
new_test_df

In [ ]:
new_test_df =  new_test_df.drop([0,1,2,3,4])
new_test_df

In [ ]:
new_test_df_ = pd.DataFrame(columns=['id', 'text', 'cleaned_summary', 'new_discourse_text'])
new_test_df_

In [ ]:
new_test_df__ = pd.concat([new_test_df_, new_test_df[['id', 'text', 'cleaned_summary', 'new_discourse_text']]])
new_test_df___ = new_test_df__.reset_index(drop=True)
new_test_df___

In [ ]:
cols = ['cleaned_summary','new_discourse_text']
new_test_df___['text_concat'] = new_test_df___[cols].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)
new_test_df___

In [ ]:
test_text = new_test_df___['text_concat'].values

In [ ]:
test_examples = convert_text_to_examples(test_text, val_label)

In [ ]:
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [ ]:
prediction = model.predict([test_input_ids, test_input_masks, test_segment_ids], verbose = 1)


In [ ]:
preds = label_encoder.classes_[np.argmax(prediction, axis =1)]

In [ ]:
submission_df = pd.DataFrame()

In [ ]:
submission_df['id'] = new_test_df___['id']

In [ ]:
submission_df

In [ ]:
submission_df['class'] = preds

In [ ]:
submission_df

In [ ]:
TEST_PATH = "test"

def get_test_text(a_id):
    a_file = f"{TEST_PATH}/{a_id}.txt"
    with open(a_file, "r") as fp:
        txt = fp.read()
    return txt

def create_df_test():
    test_ids = [f[:-4] for f in os.listdir(TEST_PATH)] #Remove the last 4 characters ('.txt') in the filenames such as '0FB0700DAF44.txt'.
    test_data = []
    for test_id in test_ids:
        text = get_test_text(test_id)
        sentences = nltk.sent_tokenize(text)
        id_sentences = []
        idx = 0 
        for sentence in sentences:
            id_sentence = []
            words = sentence.split()
            # I created this heuristic for mapping words in sentences to "word indices"
            # This is not definitive and might have strong drawbacks and problems
            for w in words:
                id_sentence.append(idx)
                idx+=1
            id_sentences.append(id_sentence)
        test_data += list(zip([test_id] * len(sentences), sentences, id_sentences))
    df_test = pd.DataFrame(test_data, columns=['id', 'discourse_text', 'ids'])
    return df_test

In [ ]:
df_test = create_df_test()
df_test.head()

In [ ]:
df_test['predictionstring'] = df_test['ids'].apply(lambda x: ' '.join([str(i) for i in x]))
df_test.head()

In [ ]:
df_test['predictionstring']

In [ ]:
submission_df['predictionstring'] = df_test['predictionstring']
submission_df

In [ ]:
submission_df[100:120]

In [ ]:
#pd.DataFrame(submission_df).to_csv('submission_result.csv', index_label='id')

In [ ]:
# submission_confirm = pd.read_csv('../input/submission-result/submission_result.csv')
# submission_confirm

In [ ]:
pd.DataFrame(submission_df).to_csv('submit.csv', index_label='id')